# Agent-based bacteria colony simulation

In [ ]:
# To - do

# multiplication of bacteria
# eating food by bacteria
# chart with bacteria growth with time
# random moving of bacteria

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

In [2]:

class Bacterium:
    def __init__(self):  # 1 - young, 2 - ready to divide, 3 - dead
        
        self.state = 1
        self.hunger_level = 0 # if hunger_level == 10 -> dies
        self.can_use_galactose = 0
        self.maturity = 0
        
    def get_hungry(self, speed=1):
        self.hunger_level += 1*speed
    
    def learn_to_use_galactose(self):
        if self.hunger_level > 6:
            self.can_use_galactose = 1
            
    def mature(self):
        if self.hunger_level < 5:
            self.maturity +=1
            
    def check_if_ready_to_divide(self, is_bacteriostatic):
        if self.maturity > 4 and is_bacteriostatic == 0:
            self.state = 2
            
    def after_division(self): # we use it for 2 cells that are created after division
        self.state = 1
        self.hunger_level = 2
        self.can_use_galactose = 0
        self.maturity = 0
    
    def check_if_die(self, is_antibiotic):
        if is_antibiotic == 1 or self.hunger == 10:
            self.state = 3 # bacteria is dead
    
    def getColour(self):
        if self.state == 1:
            return 255
        elif self.state == 2:
            return 200
        else:
            return 150  # for dead cells
    
    def move(self):
        return

    

In [3]:
class Food:
    def __init__(self, FoodType):
        self.FoodType = FoodType   # 1 - glucose, 2 - galactose, 0 - no food
        
    def getColour(self):
        if self.FoodType == 0:   # no food
            return 0
        elif self.FoodType == 1:   # glucose - light blue
            return 100
        elif self.FoodType == 2:   #galactose - dark blue
            return 50
        
    def __repr__(self):
        return str(self.FoodType)
        

In [25]:
class Board:
    
    def __init__(self, xSize, ySize):
       
        self.xSize = xSize
        self.ySize = ySize
        self.cells = []
             
        for i in range(0, xSize):
            row = []
            for j in range(0, ySize):
                row.append(Food(0))
            self.cells.append(row)
        
        self.randomBoard()

        """
              
        # initialize the board with a few food items (glucose - 1 and galactose - 2)
        
        self.food_items[1][3] = Food(1)
        self.food_items[3][7] = Food(2)
        
        
        #we only initialize bacteria where there is no food - Food with 0
        self.bacteria = []
               
        #create list of coordinates that are available for bacteria
        
        available_for_bacteria = []
        
        for i in range(0, xSize):
            for j in range(0, ySize):
                if self.food_items[i][j].FoodType == 0:
                    available_for_bacteria.append([i,j])
                    
        bacteria_location1 = available_for_bacteria[1]
        bacteria_location2 = available_for_bacteria[5]
        
        
        # create bacteria on empty spaces
        
        self.food_items[bacteria_location1[0]][bacteria_location1[1]] = Bacterium()
        self.food_items[bacteria_location2[0]][bacteria_location2[1]] = Bacterium()
        """
                
    def printBoard(self):
        print('printing the board')
        print(self.cells)
                                                    
    def randomBoard(self):  #TODO - initializing a random board with food
      
        # random boolean mask    
        fields_mask = np.random.choice([0,1,2,3],size=(self.xSize,self.ySize), p=[0.3,0.2,0.2,0.3])

        for i in range (0,len(fields_mask)-1):
            for j in range (0,len(fields_mask[0])-1):
                if fields_mask[i][j] == 1:
                    self.cells[i][j] = Food(1)
                elif fields_mask[i][j] == 2:
                    self.cells[i][j] = Food(2)
                elif fields_mask[i][j] == 3:
                    self.cells[i][j] = Bacterium()
                    
        return

        
    def iteration(self): 
        #print('iteration!')
        self.printBoard()

        for i in range(1,len(self.cells)-1):
            for j in range(1, len(self.cells[0])-1):
              if isinstance(self.cells[i][j], Bacterium):
                self.cells[i][j].move()
                self.cells[i][j].check_if_die()
                self.cells[i][j].check_if_ready_to_divide()
                if self.cells[i][j].state == 2:
                    self.cells[i][j].after_division()
                    # Moore
                    tmpList = [self.cells[i-1][j-1], self.cells[i-1][j], self.cells[i-1][j+1],
                                self.cells[i][j-1], self.cells[i][j+1], self.cells[i+1][j-1], 
                               self.cells[i+1][j], self.cells[i+1][j+1]]
                    for cell in tmpList:
                        if isinstance(cell, Food): 
                            if cell.FoodType == 0:
                                cell = Bacterium
                                cell.after_division()

    
    def clear(self):
        """
        for i in range(0,len(self.food_items)):
            for j in range(0, len(self.food_items[0])):
                self.food_items[i][j].clear_static()
        #print('robie clear booard')
        self.calculateField()
        """
    
    def redraw(self):
        # Returns numpy array of colours
        print('redraws!')
        fig = np.zeros((len(self.cells),len(self.cells[0])))
        for i in range(0,len(self.cells)):
            for j in range(0, len(self.cells[0])):
                fig[i][j] = self.cells[i][j].getColour()
        print('fig:', fig)
        return fig
    
    
                
            

In [16]:
def update(frameNum, board, im, xSize, ySize):
    if(frameNum == 0):
        board.randomBoard(xSize, ySize)       
        im.set_data(board.redraw()) 
    else:   
        board.iteration()
        #print('field map: ', board.getFieldMap())
        im.set_data(board.redraw()) 
    return im,

In [17]:
%%capture
# Parameters
steps = 10
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [24]:
#Run Simulation
import random 

xSize = 10
ySize= 10
board = Board(xSize,ySize)
board.printBoard()
board.clear()

#im = ax.imshow(board.redraw(),  vmin=0, vmax=255)
#ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, im, xSize, ySize])
#HTML(ani.to_jshtml())


printing the board
[[<__main__.Bacterium object at 0x7f42aaccb950>, 2, <__main__.Bacterium object at 0x7f42aaccb2d0>, <__main__.Bacterium object at 0x7f42aaccbc90>, 2, <__main__.Bacterium object at 0x7f42aaccbe50>, 2, 2, 0, 0], [1, 0, <__main__.Bacterium object at 0x7f42aaccbd50>, 1, <__main__.Bacterium object at 0x7f42aaccbf50>, <__main__.Bacterium object at 0x7f42aaccb350>, 1, 1, 2, 0], [2, 1, 1, 0, 1, 2, 0, <__main__.Bacterium object at 0x7f42aaccb410>, <__main__.Bacterium object at 0x7f42aaccb110>, 0], [<__main__.Bacterium object at 0x7f42aaccbdd0>, <__main__.Bacterium object at 0x7f42aaccb1d0>, <__main__.Bacterium object at 0x7f42aaccb750>, 1, 1, <__main__.Bacterium object at 0x7f42aaccbf90>, 1, <__main__.Bacterium object at 0x7f42aaccb910>, 2, 0], [<__main__.Bacterium object at 0x7f42aaccba90>, 0, 2, <__main__.Bacterium object at 0x7f42aaccbe90>, 2, 0, <__main__.Bacterium object at 0x7f42aaccb210>, 1, 1, 0], [<__main__.Bacterium object at 0x7f42aaccb4d0>, 1, 1, 0, <__main__.Bacte